In [32]:
## данная функция по линку rabota.ua выдает таблицу с полями 
##'link_1','предприятие','должность','период','отрасль','duration'
## за якори я принимала тэги html , поэтому много ошибок в 'предприятие','должность'
## в полях  'период','отрасль','duration' ошибок почти нет

def duration(link):
    from bs4 import BeautifulSoup
    import requests
    import urllib.request
    import pandas as pd
    import re
    import warnings
    warnings.filterwarnings('ignore') # полезно чтобы избежать ругательства при создании ного поля таблицы(почемуто все-время ругня)
    
    #стандартный парсинг страницы
    response=requests.get(link)
    html_contents = response.text
    html_soup = BeautifulSoup(html_contents,'html.parser')
    
    # парсим в одном контейнере 'div',id='ExperienceHolder'
    # для этого обязательно в одну строчку делаем цепочку find(...).find(...)
    # и только потом складываем это в лист циклом for
        
    gde=[]
    kogda=[]
    b1=html_soup.find('div',id='ExperienceHolder').find_all('b')
    m=html_soup.find('div',id='ExperienceHolder').find_all('em')
    for i in b1:
        gde.append(i.text.strip())
    for k in m:
        kogda.append(k.text.strip())
    
    # если слово выделено, то оно попадает в gde, там образуется шлак
    # попыталась сделать ручные исключения
    # работает плохо, но как попытка
    
    gde=[x for x in gde if "Достиж" not in x]
    gde=[x for x in gde if "Рекоменд" not in x]
    gde=[x for x in gde if "улучш" not in x]
    gde=[x for x in gde if "Результ" not in x]
    gde=[x for x in gde if "Обязанн" not in x]
    gde=[x for x in gde if "достиж" not in x]
    gde=gde[:min(len(gde),len(kogda))] # привожу два листа к одной размерности
    
    kogda=kogda[:min(len(gde),len(kogda))]# привожу два листа к одной размерности
    d = {'должность':gde,'период':kogda} # создаю словать и dataframe
    uh=pd.DataFrame(d)
    
    uh['предприятие']=uh['должность'].shift(-1)# копирую и поднимаю столбец, чтобы сделать строки
    uh['отрасль']=uh['период'].shift(-1)# копирую и поднимаю столбец, чтобы сделать строки
    
    uh['отрасль']=uh['отрасль'].str.extract('\\t(\w+)').fillna(uh['отрасль'])# все-равно не работает
    
    uhh=uh.iloc[::2, :] # беру каждую вторую строку
    
    # извлекаю из строчного периода цифровой год и месяц
    regex=re.compile('(\(\d+.лет|\(\d+.год)')
    uhh['год']=uhh['период'].str.extract(regex)
    uhh['год']=uhh['год'].str.extract(r'(\d+)').fillna(0)
    uhh['мес']=uhh['период'].str.extract('(\d+).мес').fillna(0)
    uhh['мес1']=uhh['мес'].astype(int)/12
    uhh['duration']=uhh['год'].astype(int)+uhh['мес1'].round(2)
    
    uhh['link']=link
    uhh=uhh[['link','предприятие','должность','период','отрасль','duration']].reset_index(drop=True)
    return uhh